In [5]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Flatten, Dense, Reshape
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
from keras import initializers
import keras


import numpy as np
import sys
import pickle
import matplotlib.pyplot as plt
import matplotlib
import datetime
import imageio
import cv2

In [12]:
patch_size = 56
input_patch_size = 56

In [6]:
descr_name = 'learned_brief_64_binary_datagen30'
base_dir = '/home/niaki/PycharmProjects/learned-brief/weights'
model_version = '0.1.0.0.3_whole_datagen_dense1152_conv32_conv32_up_conv32_conv32_up_conv32_conv8_conv1__relu_last_sigmoid_bce_output24X24_increasedgeometricnoise'

In [7]:
brief_ae = load_model(base_dir + '/brief_ae_' + model_version + '.h5')

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


In [13]:
input_shape = (input_patch_size, input_patch_size, 1)
input_img = Input(shape=input_shape)

x = Conv2D(1, (3,3), activation="relu", padding="valid")(input_img)
x = Conv2D(1, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(1, (3, 3), activation="relu", padding="valid")(x)
average_blurred = Conv2D(1, (3, 3), activation="relu", padding="valid")(x)
flattened = Flatten(data_format="channels_last")(average_blurred)
briefed = Dense(512, activation="sigmoid")(flattened)

brief_encoder = Model(input_img, briefed)

for i in range(len(brief_encoder.layers)):
    brief_encoder.get_layer(index=i).set_weights(brief_ae.get_layer(index=i).get_weights())

brief_encoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 56, 56, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 54, 54, 1)         10        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 1)         10        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 1)         10        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 48, 48, 1)         10        
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1180

In [15]:
brief_encoder.compile(optimizer="adadelta", loss="binary_crossentropy")
brief_encoder.save(base_dir + '/brief_encoder.h5') # from this model: model_version = '0.1.0.0.3_whole_datagen_dense1152_conv32_conv32_up_conv32_conv32_up_conv32_conv8_conv1__relu_last_sigmoid_bce_output24X24_increasedgeometricnoise'

In [9]:
descr_name = 'opencv_brief'
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create(64, False)

In [10]:
keypoint = cv2.KeyPoint((patch_size - 1) / 2, (patch_size - 1) / 2, _size=brief_patch_size)
keypoints = [keypoint]

NameError: name 'brief_patch_size' is not defined